<a href="https://colab.research.google.com/github/Itssafa/NewProject/blob/main/claim_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.inspection import permutation_importance
import joblib
import warnings
warnings.filterwarnings('ignore')

# 1. Chargement des données
print("1. Chargement et exploration des données")

df = pd.read_csv('claims_dataset.csv')


print(f"Dimensions du dataset: {df.shape}")
print("\nAperçu des données:")
print(df.head())

# Affichage des statistiques descriptives
print("\nStatistiques descriptives:")
print(df.describe())

# Vérification des valeurs manquantes
print("\nValeurs manquantes par colonne:")
print(df.isnull().sum())

# 2. Prétraitement des données
print("\n2. Prétraitement des données")

# Supposons que la variable cible est 'urgency_classification'
target = 'urgency_classification'
print(f"Variable cible: {target}")
print(f"Distribution de la variable cible:\n{df[target].value_counts()}")

# Identification des types de variables
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns.tolist()
categorical_cols.remove(target) if target in categorical_cols else None
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nVariables catégorielles: {categorical_cols}")
print(f"Variables numériques: {numeric_cols}")

# Création des ensembles d'entraînement et de test
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nDimensions de l'ensemble d'entraînement: {X_train.shape}")
print(f"Dimensions de l'ensemble de test: {X_test.shape}")

# Préprocesseur pour transformer les données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# 3. Création et entraînement des modèles
print("\n3. Entraînement et évaluation des modèles")

# Définition des modèles à tester
models = {
    'Régression Logistique': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Stockage des résultats
results = {}

# Validation croisée et entraînement des modèles
print("\nRésultats de la validation croisée (5-fold):")
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

    # Validation croisée
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{name}: Accuracy = {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

    # Entraînement sur l'ensemble complet d'entraînement
    pipeline.fit(X_train, y_train)

    # Prédictions sur l'ensemble de test
    y_pred = pipeline.predict(X_test)

    # Calcul des métriques
    results[name] = {
        'model': pipeline,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted'),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }

# 4. Évaluation des performances
print("\n4. Benchmarking des modèles")
print("\nPerformances sur l'ensemble de test:")

# Tableau comparatif des performances
perf_df = pd.DataFrame({
    'Modèle': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results],
    'Precision': [results[m]['precision'] for m in results],
    'Recall': [results[m]['recall'] for m in results],
    'F1-Score': [results[m]['f1'] for m in results]
})

print(perf_df.sort_values('F1-Score', ascending=False))

# Identification du meilleur modèle (basé sur F1-score)
best_model_name = perf_df.sort_values('F1-Score', ascending=False).iloc[0]['Modèle']
best_model = results[best_model_name]['model']
print(f"\nMeilleur modèle: {best_model_name}")

# 5. Analyse détaillée du meilleur modèle
print("\n5. Analyse détaillée du meilleur modèle")

# Rapport de classification détaillé
y_pred_best = best_model.predict(X_test)
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_best))

# 6. Tests de robustesse
print("\n6. Tests de robustesse")

# Test avec des données bruitées
print("\nTest avec des données bruitées:")
X_test_noisy = X_test.copy()
for col in numeric_cols:
    if col in X_test_noisy.columns:
        # Ajouter du bruit gaussien (10% de l'écart-type)
        noise = np.random.normal(0, 0.1 * X_test_noisy[col].std(), size=X_test_noisy[col].shape)
        X_test_noisy[col] = X_test_noisy[col] + noise

y_pred_noisy = best_model.predict(X_test_noisy)
print(f"Accuracy avec données bruitées: {accuracy_score(y_test, y_pred_noisy):.4f}")
print(f"Différence d'accuracy: {accuracy_score(y_test, y_pred_best) - accuracy_score(y_test, y_pred_noisy):.4f}")

# 7. Analyse des erreurs
print("\n7. Analyse des erreurs")
y_proba = best_model.predict_proba(X_test)
errors = X_test.iloc[np.where(y_pred_best != y_test)[0]]
error_predictions = y_pred_best[np.where(y_pred_best != y_test)[0]]
error_truth = y_test.iloc[np.where(y_pred_best != y_test)[0]]

print(f"Nombre total d'erreurs: {len(errors)} sur {len(X_test)} échantillons de test ({len(errors)/len(X_test)*100:.2f}%)")

if len(errors) > 0:
    error_df = pd.DataFrame({
        'Prédiction': error_predictions,
        'Réalité': error_truth
    })
    print("\nDistribution des types d'erreurs:")
    print(error_df.groupby(['Réalité', 'Prédiction']).size().reset_index(name='count'))

# 8. Courbes d'apprentissage
print("\n8. Courbes d'apprentissage")
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy', n_jobs=-1)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

print(f"Score d'entraînement final: {train_mean[-1]:.4f}")
print(f"Score de validation final: {test_mean[-1]:.4f}")
print(f"Différence (overfitting): {train_mean[-1] - test_mean[-1]:.4f}")

# 9. Sauvegarde du modèle
print("\n9. Sauvegarde du modèle pour implémentation")
# Sauvegarde du meilleur modèle
joblib.dump(best_model, 'best_claims_model.pkl')
print("Modèle sauvegardé sous 'best_claims_model.pkl'")

# 10. Simulation de déploiement
print("\n10. Simulation d'utilisation en production")

def predict_claim_priority(claim_data):
    """
    Fonction pour simuler l'utilisation du modèle en production
    Args:
        claim_data: Dictionnaire contenant les données d'une réclamation
    Returns:
        Classe prédite et probabilités
    """
    # Conversion en DataFrame
    claim_df = pd.DataFrame([claim_data])

    # Prédiction
    prediction = best_model.predict(claim_df)[0]
    probabilities = best_model.predict_proba(claim_df)[0]

    return prediction, probabilities

# Exemple d'une nouvelle réclamation
print("\nTest avec un exemple de nouvelle réclamation:")
# Utilisez un exemple réel de votre dataset
new_claim = X_test.iloc[0].to_dict()
predicted_class, probas = predict_claim_priority(new_claim)
print(f"Classe prédite: {predicted_class}")
print(f"Probabilités: {probas}")
print(f"Classe réelle: {y_test.iloc[0]}")

print("\nConclusion: Le modèle est prêt pour les tests en environnement de pré-production")
print("Prochaines étapes recommandées:")
print("1. Configurer un environnement de pré-production")
print("2. Exécuter des tests A/B comparant le modèle avec l'approche actuelle")
print("3. Surveiller les performances sur un sous-ensemble de données réelles")
print("4. Préparer une stratégie de mise à jour et maintenance du modèle")

1. Chargement et exploration des données
Dimensions du dataset: (2000, 12)

Aperçu des données:
        sender    receiver  number_of_characters  contains_urgent  \
0  Electrician   Architect                   366            False   
1    Architect   Architect                    44            False   
2   Mechanical   Architect                    38            False   
3     Engineer  Contractor                   102            False   
4  Electrician   Architect                   142             True   

   contains_important        urgency_classification   send_date  \
0               False  not urgent and not important  2016-07-10   
1               False  not urgent and not important  2016-11-11   
2               False  not urgent and not important  2019-10-11   
3               False  not urgent and not important  2020-03-06   
4               False          urgent and important  2016-11-10   

             send_time      claim_type initial_priority  \
0  2016-07-10 17:19:31  adm

In [ ]:

import joblib

# Sauvegarde du meilleur modèle
joblib.dump(models[best_model_name], 'urgent_model.pkl')
print("✅ Modèle sauvegardé sous 'urgent_model.pkl'")

✅ Modèle sauvegardé sous 'urgent_model.pkl'
